In [ ]:
import tensorflow as tf
import numpy as np
import os

from keras.models import Model
from keras.optimizers import *
from keras.layers import *

In [ ]:
def Segnet(input_size=(360, 480, 3)):
    inputs = Input(input_size)
    
    conv1 = conv_relu(inputs, 64)
    conv1 = conv_relu_pool(conv1, 64)
    
    conv2 = conv_relu(conv1, 128)
    conv2 = conv_relu_pool(conv2, 128)
    
    conv3 = conv_relu(conv2, 256)
    conv3 = conv_relu(conv3, 256)
    conv3 = conv_relu_pool(conv3, 256)
    
    conv4 = conv_relu(conv3, 512)
    conv4 = conv_relu(conv4, 512)
    conv4 = conv_relu_pool(conv4, 512)
    
    conv5 = conv_relu(conv4, 512)
    conv5 = conv_relu(conv5, 512)
    conv5 = conv_relu_pool(conv5, 512)
    
    upsample1 = upsample(conv5)
    conv6 = conv_relu(upsample1, 512)
    conv6 = conv_relu(conv6, 512)
    conv6 = conv_relu(conv6, 512)
    
    upsample2 = upsample(conv6)
    conv7 = conv_relu(upsample2, 512)
    conv7 = conv_relu(conv7, 512)
    conv7 = conv_relu(conv7, 512)
    
    upsample3 = upsample(conv7)
    conv8 = conv_relu(upsample3, 256)
    conv8 = conv_relu(conv8, 256)
    conv8 = conv_relu(conv8, 256)
    
    upsample4 = upsample(conv8)
    conv9 = conv_relu(upsample4, 128)
    conv9 = conv_relu(conv9, 128)
    
    upsample5 = upsample(conv9)
    conv10 = conv_relu(upsample5, 64)
    conv10 = conv_relu(conv10, 64)
    
    out = Softmax() (conv10)
    
    model = Model(input=inputs, output=out)
    model.compile(optimizer = SGD(lr=0.1, momentum=0.9), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model
    

In [ ]:
# Helper functions

def conv_relu(image, num_filters, ksize=3, stride=1, pad="SAME"):
    
    conv = Conv2D(filters=num_filters, kernel_size=ksize, 
                            strides=stride, padding=pad, activation='relu', 
                            kernel_initializer='he_normal') (image)
    
    conv = BatchNormalization() (conv)
    return conv


def conv_relu_pool(image, num_filters, poolsize=2, pool_stride=2, pool_pad="VALID"):
    conv = conv_relu(image, num_filters)
    pool = MaxPool2D(pool_size=poolsize, strides=pool_stride, padding=pool_pad) (conv)
    return pool


def upsample(image):
    # Default upsample scale is (2,2)
    return UpSampling2D() (image)

In [ ]:
model = Segnet()